## Load the data

In [2]:
import pandas as pd
books=pd.read_csv('bookstore_transactions.csv')
print(books.head())

          Transaction
0    History,Bookmark
1    History,Bookmark
2    Fiction,Bookmark
3  Biography,Bookmark
4    History,Bookmark


In [62]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Transaction   99 non-null     object
 1   transactions  99 non-null     object
dtypes: object(2)
memory usage: 1.7+ KB


In [47]:
retail=pd.read_csv('online_retail.csv')
print(retail.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227760 entries, 0 to 227759
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   InvoiceNo    227760 non-null  object
 1   StockCode    227760 non-null  object
 2   Description  227404 non-null  object
dtypes: object(3)
memory usage: 5.2+ MB
None


In [63]:
movie=pd.read_csv('movielens_movies.csv')
print(movie.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [48]:
print(retail.head())

  InvoiceNo StockCode                          Description
0    562583    35637A      IVORY STRING CURTAIN WITH POLE 
1    562583    35638A        PINK AND BLACK STRING CURTAIN
2    562583    84927F                PSYCHEDELIC TILE HOOK
3    562583     22425                ENAMEL COLANDER CREAM
4    562583     16008  SMALL FOLDING SCISSOR(POINTED EDGE)


In [14]:
transactions=books['Transaction'].apply(lambda t: t.split(','))

In [17]:
transactions=list(transactions)

In [18]:
transactions.count(['biography','fiction'])

0

## identifying assocaitions rules with itertools

In [21]:
from itertools import permutations

In [41]:
import numpy as np
list2=[]
for item in transactions:
    list2.extend(item)
print(set(list2))

{'Poetry', 'History', 'Biography', 'Fiction', 'Bookmark'}


In [43]:
# compute and print rules
rules=list(permutations(set(list2),2))
print(rules)

[('Poetry', 'History'), ('Poetry', 'Biography'), ('Poetry', 'Fiction'), ('Poetry', 'Bookmark'), ('History', 'Poetry'), ('History', 'Biography'), ('History', 'Fiction'), ('History', 'Bookmark'), ('Biography', 'Poetry'), ('Biography', 'History'), ('Biography', 'Fiction'), ('Biography', 'Bookmark'), ('Fiction', 'Poetry'), ('Fiction', 'History'), ('Fiction', 'Biography'), ('Fiction', 'Bookmark'), ('Bookmark', 'Poetry'), ('Bookmark', 'History'), ('Bookmark', 'Biography'), ('Bookmark', 'Fiction')]


In [45]:
print(len(rules))

20


## Metrics and pruning
A metric is measure of performance for rules
Pruning is the use of meters to discard rules

In [51]:
from mlxtend.preprocessing import TransactionEncoder

In [52]:
encoder=TransactionEncoder().fit(transactions)

In [54]:
print(encoder)

TransactionEncoder()


In [53]:
onehot=encoder.transform(transactions)

In [69]:
retail

,InvoiceNo,StockCode,Description
0,562583,35637A,IVORY STRING CURTAIN WITH POLE
1,562583,35638A,PINK AND BLACK STRING CURTAIN
2,562583,84927F,PSYCHEDELIC TILE HOOK
3,562583,22425,ENAMEL COLANDER CREAM
4,562583,16008,SMALL FOLDING SCISSOR(POINTED EDGE)
...,...,...,...
227755,C581229,23158,SET OF 5 LUCKY CAT MAGNETS
227756,C581229,22712,CARD DOLLY GIRL
227757,C581229,22027,TEA PARTY BIRTHDAY CARD
227758,C581229,21508,VINTAGE KID DOLLY CARD


In [57]:
onehot=pd.DataFrame(onehot,columns=encoder.columns_)

In [58]:
print(onehot.mean())

Biography    0.404040
Bookmark     1.000000
Fiction      0.252525
History      0.252525
Poetry       0.090909
dtype: float64


## Confidence metrics
can imporve over support with additional metrics
add confidence provides a more complete picture
Support(milk&coffee)/Support(milk)
## The life metric
Support(X&Y)/Support(X)*Support(Y)

In [ ]:
# Compute support for Hunger and Potter
supportHP = np.logical_and(books['Hunger'], books['Potter']).mean()

# Compute support for Hunger and Twilight
supportHT = np.logical_and(books['Hunger'], books['Twilight']).mean()

# Compute support for Potter and Twilight
supportPT = supportHP / supportHT 

# Print support values
print("Hunger Games and Harry Potter: %.2f" % supportHP)
print("Hunger Games and Twilight: %.2f" % supportHT)
print("Harry Potter and Twilight: %.2f" % supportPT)

In [ ]:
# Compute support for Potter and Twilight
supportPT = np.logical_and(books['Potter'],books['Twilight']).mean()

# Compute support for Potter
supportP = books['Potter'].mean()

# Compute support for Twilight
supportT = books['Twilight'].mean()

# Compute confidence for both rules
confidencePT = supportPT / supportP
confidenceTP = supportPT / supportT

# Print results
print('{0:.2f}, {1:.2f}'.format(confidencePT, confidenceTP))

## Building on simpler metrics
support(X)=frequency(X)/N
support(X-Y)=frequency(X&y)/N
confidence(X-y)=support(x-y)/support(x)
lift(x-y)=support(x-y)/support(X)support(y)
** good rule is 1
the leverage metric
leverage(x-y)=support(x-y)-support(X)support(y)
** good rule is 0
Conviction(x-y)= support(X)support(y)/support(x-y)

In [ ]:
# Compute support for Potter AND Hunger
supportPH = np.logical_and(books['Potter'], books['Hunger']).mean()

# Compute support for Potter
supportP = books['Potter'].mean()

# Compute support for NOT Hunger
supportnH = 1.0 - books['Hunger'].mean()

# Compute support for Potter and NOT Hunger
supportPnH = supportP - supportPH

# Compute and print conviction for Potter -> Hunger
conviction = supportP * supportnH / supportPnH
print("Conviction: %.2f" % conviction)

In [ ]:
def conviction(antecedent, consequent):
	# Compute support for antecedent AND consequent
	supportAC = np.logical_and(antecedent, consequent).mean()

	# Compute support for antecedent
	supportA = antecedent.mean()

	# Compute support for NOT consequent
	supportnC = 1.0 - consequent.mean()

	# Compute support for antecedent and NOT consequent
	supportAnC = supportA - supportAC

    # Return conviction
	return supportA * supportnC / supportAnC

# Compute conviction for twilight -> potter and potter -> twilight
convictionTP = conviction(twilight, potter)
convictionPT = conviction(potter, twilight)

# Compute conviction for twilight -> hunger and hunger -> twilight
convictionTH = conviction(twilight, hunger)
convictionHT = conviction(hunger, twilight)

# Compute conviction for potter -> hunger and hunger -> potter
convictionPH = conviction(potter, hunger)
convictionHP = conviction(hunger, potter)

# Print results
print('Harry Potter -> Twilight: ', convictionHT)
print('Twilight -> Potter: ', convictionTP)

## Association and Disassociation
## Zhang metrics
** value+1 perfect assocation
** value-1 perfect disassociations

In [ ]:
# Compute the support of Twilight and Harry Potter
supportT = books['Twilight'].mean()
supportP = books['Potter'].mean()
# Compute the support of both books
supportTP = np.logical_and(books['Twilight'],books['Potter']).mean()

# Complete the expressions for the numerator and denominator
numerator = supportTP - supportT*supportP
denominator = max(supportTP*(1-supportT), supportT*(supportP-supportTP))

# Compute and print Zhang's metric
zhang = numerator / denominator
print(zhang)

In [ ]:
# Define a function to compute Zhang's metric
def zhang(antecedent, consequent):
	# Compute the support of each book
	supportA = antecedent.mean()
	supportC = consequent.mean()

	# Compute the support of both books
	supportAC = np.logical_and(antecedent, consequent).mean()

	# Complete the expressions for the numerator and denominator
	numerator = supportAC - supportA*supportC
	denominator = max(supportAC*(1-supportA), supportA*(supportC-supportAC))

	# Return Zhang's metric
	return numerator / denominator

In [ ]:
# Define an empty list for Zhang's metric
zhangs_metric = []

# Loop over lists in itemsets
for itemset in itemsets:
    # Extract the antecedent and consequent columns
	antecedent = books[itemset[0]]
	consequent = books[itemset[1]]
    
    # Complete Zhang's metric and append it to the list
	zhangs_metric.append(zhang(antecedent, consequent))
    
# Print results
rules['zhang'] = zhangs_metric
print(rules)

## Advanced 
1. generate large set of rules

2. perform muti-metric filtering

In [ ]:
# Preview the rules DataFrame using the .head() method
print(rules.head())

# Select the subset of rules with antecedent support greater than 0.05
rules = rules[rules['antecedent support'] >0.05]

# Select the subset of rules with a consequent support greater than 0.01
rules = rules[rules['consequent support']>0.01]

# Select the subset of rules with a conviction greater than 1.01
rules = rules[rules['conviction']>1.01]

# Print remaining rules
print(rules)

## Aggregation
items_categories

In [71]:
retail.head()

,InvoiceNo,StockCode,Description
0,562583,35637A,IVORY STRING CURTAIN WITH POLE
1,562583,35638A,PINK AND BLACK STRING CURTAIN
2,562583,84927F,PSYCHEDELIC TILE HOOK
3,562583,22425,ENAMEL COLANDER CREAM
4,562583,16008,SMALL FOLDING SCISSOR(POINTED EDGE)


In [72]:
invoice=retail['InvoiceNo'].unique()

In [73]:
libraries=[list(retail[retail['InvoiceNo']==u].Description) for u in invoice]

In [74]:
print(libraries[0])

['IVORY STRING CURTAIN WITH POLE ', 'PINK AND BLACK STRING CURTAIN', 'PSYCHEDELIC TILE HOOK', 'ENAMEL COLANDER CREAM', 'SMALL FOLDING SCISSOR(POINTED EDGE)', 'JIGSAW TOADSTOOLS 3 PIECE']


In [67]:
print(len(retail['Description'].unique()))

3461


## pruning

In [ ]:
def aggregate(item):
	# Select the column headers for sign items
	item_headers = [i for i in onehot.columns if i.lower().find(item)>=0]

	# Select columns of sign items
	item_columns = onehot[item_headers]

	# Return category of aggregated items
	return item_columns.sum(axis = 1) >= 1.0

# Aggregate items for the bags, boxes, and candles categories  
bags = aggregate('bag')
boxes = aggregate('boxes')
candles = aggregate('candles')

## Aprior algorithm
**Reducing the number of itemsets
** sensible
- subsets of frequent sets are frequent
- retain sets known to be frequent
- prune sets not known to be frequent

In [ ]:
# Import apriori from mlxtend
from mlxtend.frequent_patterns import apriori

print(onehot)
# Compute frequent itemsets using the Apriori algorithm
frequent_itemsets = apriori(onehot, 
                            min_support = 0.006, 
                            max_len = 3, 
                            use_colnames = True)

# Print a preview of the frequent itemsets
print(frequent_itemsets.head())

In [ ]:
# Import apriori from mlxtend
from mlxtend.frequent_patterns import apriori

# Compute frequent itemsets using a support of 0.003 and length of 3
frequent_itemsets_1 = apriori(onehot, min_support = 0.003, 
                            max_len = 3, use_colnames = True)

# Compute frequent itemsets using a support of 0.001 and length of 3
frequent_itemsets_2 = apriori(onehot, min_support = 0.001, 
                           max_len=3, use_colnames = True)

# Print the number of freqeuent itemsets
print(len(frequent_itemsets_1), len(frequent_itemsets_2))

## Apriori pruning
### Prunes itemsets
applies minimum support threhold
modified version can prun by number of items
Doesn't tell us about association rules
### association rules
many more assocaition rules than itemsets

## Generating association rules

In [ ]:
# Import the association rule function from mlxtend
from mlxtend.frequent_patterns import association_rules

# Compute all association rules for frequent_itemsets_1
rules_1 = association_rules(frequent_itemsets_1, 
                            metric = "support", 
                         	min_threshold = 0.0015)

frequent_itemsets_1

# Compute all association rules for frequent_itemsets_2
rules_2 = association_rules(frequent_itemsets_2, 
                            metric = "support", 
                        	min_threshold=0.0015)

# Print the number of association rules generated
print(len(rules_1), len(rules_2))

## Pruning with lift

In [ ]:
# Import the association rules function
from mlxtend.frequent_patterns import association_rules

# Compute frequent itemsets using the Apriori algorithm
frequent_itemsets = apriori(onehot, min_support =0.001, 
                            max_len = 2, use_colnames = True)

# Compute all association rules for frequent_itemsets
rules = association_rules(frequent_itemsets, 
                            metric = "lift", 
                         	min_threshold = 1)

# Print association rules
print(rules)

## Pruning with confidence

In [ ]:
# Import the association rules function
from mlxtend.frequent_patterns import association_rules

# Compute frequent itemsets using the Apriori algorithm
frequent_itemsets = apriori(onehot, min_support=0.0015, 
                            max_len=2, use_colnames = True)

# Compute all association rules using confidence
rules = association_rules(frequent_itemsets, 
                            metric = "confidence", 
                         	min_threshold = 0.5)

# Print association rules
print(rules)

## Aggregation and filtering

In [ ]:
# Apply the apriori algorithm with a minimum support of 0.0001
frequent_itemsets = apriori(aggregated, min_support=0.0001, use_colnames = True)

# Generate the initial set of rules using a minimum support of 0.0001
rules = association_rules(frequent_itemsets, 
                          metric = "support", min_threshold = 0.001)

# Set minimum antecedent support to 0.35
rules = rules[rules['antecedent support'] > 0.35]

# Set maximum consequent support to 0.35
rules = rules[rules['consequent support']  < 0.35]

# Print the remaining rules
print(rules)

## Applying Zhang's rule

In [ ]:
# Generate the initial set of rules using a minimum lift of 1.00
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1)

# Set antecedent support to 0.005
rules = rules[rules['antecedent support'] > 0.005]

# Set consequent support to 0.005
rules = rules[rules['consequent support'] > 0.005]

# Compute Zhang's rule
rules['zhang'] = zhangs_rule(rules)

# Set the lower bound for Zhang's rule to 0.98
rules = rules[rules['zhang'] > 0.98]
print(rules[['antecedents', 'consequents']])

## Advanced filtering with multiple metrics

In [ ]:
# Apply the Apriori algorithm with a minimum support threshold of 0.001
frequent_itemsets = apriori(onehot, min_support = 0.001, use_colnames = True)

# Recover association rules using a minium support threshold of 0.001
rules = association_rules(frequent_itemsets, metric = 'support', min_threshold = 0.001)

# Apply a 0.002 antecedent support threshold, 0.60 confidence threshold, and 2.50 lift threshold
filtered_rules = rules[(rules['antecedent support'] > 0.002) &
						(rules['consequent support'] > 0.01) &
						(rules['confidence'] > 0.6) &
						(rules['lift'] > 2.50)]

# Print remaining rule
print(filtered_rules[['antecedents','consequents']])